In [1]:
from model.clamp import CLaMPLite   

model = CLaMPLite.load_from_checkpoint('/home/yj/PycharmProjects/MIT/CLaMP/outputs/2023-10-21/03-47-05/epoch=6-step=1386.ckpt')
model.eval()

/home/yj/anaconda3/envs/llamat/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CLaMPLite(
  (graph_encoder): CGCNN(
    (embedding): Linear(in_features=92, out_features=512, bias=True)
    (rbf): RBFExpansion()
    (convs): ModuleList(
      (0-2): 3 x CGConv(512, dim=80)
    )
    (conv_to_fc): Sequential(
      (0): Linear(in_features=512, out_features=128, bias=True)
      (1): Softplus(beta=1, threshold=20)
      (2): Linear(in_features=128, out_features=512, bias=True)
    )
  )
  (text_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31116, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
 

In [5]:
text = ["I am a student.", "I am a student."]
tokenizer = model.tokenizer
input = tokenizer(text, return_tensors="pt")
input

{'input_ids': tensor([[ 101,  453,  568,  105, 6128,  211,  102],
        [ 101,  453,  568,  105, 6128,  211,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]])}

In [7]:
input.input_ids.shape, input.attention_mask.shape

(torch.Size([2, 7]), torch.Size([2, 7]))

In [6]:
model.text_encoder(input['input_ids'].to(model.device), input['attention_mask'].to(model.device))

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.4878, -0.0894, -0.4275,  ...,  0.1494, -0.3210,  1.4815],
         [-0.1404,  0.0760, -0.8578,  ...,  0.1239,  0.1869,  1.6717],
         [ 0.1667,  0.2629, -0.3731,  ...,  0.2571, -0.5682,  1.9967],
         ...,
         [-0.4988, -0.5652, -0.0850,  ...,  0.0650, -0.3613,  1.7268],
         [-0.4990, -0.1551, -0.3210,  ...,  0.2011, -0.3753,  1.5463],
         [-0.3354,  0.5048, -0.7920,  ..., -0.2029, -0.4281,  1.2430]],

        [[-0.4878, -0.0894, -0.4275,  ...,  0.1494, -0.3210,  1.4815],
         [-0.1404,  0.0760, -0.8578,  ...,  0.1239,  0.1869,  1.6717],
         [ 0.1667,  0.2629, -0.3731,  ...,  0.2571, -0.5682,  1.9967],
         ...,
         [-0.4988, -0.5652, -0.0850,  ...,  0.0650, -0.3613,  1.7268],
         [-0.4990, -0.1551, -0.3210,  ...,  0.2011, -0.3753,  1.5463],
         [-0.3354,  0.5048, -0.7920,  ..., -0.2029, -0.4281,  1.2430]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackw

In [8]:
import pandas as pd

df = pd.read_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_materials_graphs_gpt_questions_test.parquet')
df

,atoms,space group symbol,crystal system,energy per atom,volume,formation energy per atom,pretty formula,energy above hull,band gap,density,...,gpt_explanation,reduced_formula,edge_index,edge_attr,num_nodes,node_feat,y,text,structure_question_list,composition_question_list
113701,"{'abc': [3.6269, 3.6269, 3.6269], 'angles': [9...",Pm-3m,cubic,-7.257871,47.709746,-0.062245,EuCu,0.071926,0.0000,7.500832,...,One possible application area for this materia...,EuCu,"[[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,...","[[1.8134499788284302, -1.8134499788284302, 1.8...",2,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The substance with the database ID 113701 has ...,"[This material is cubic., This material is not...","[This material is composed of ['Europium', 'Co..."
113702,"{'abc': [7.965815, 7.965816, 7.96581], 'angles...",I-4m2,tetragonal,-7.887140,319.857203,-3.248899,SiO2,0.029099,5.6453,1.559639,...,The material described here is one of the most...,SiO2,"[[0, 7, 0, 5, 0, 6, 0, 8, 0, 1, 0, 2, 0, 4, 0,...","[[-0.06221536546945572, -1.628993272781372, 0....",15,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The material represented by this entry in the ...,"[This material is tetragonal., This material i...","[This material is composed of ['Oxygen', 'Sili..."
113703,"{'abc': [7.965159, 7.965154, 7.96516], 'angles...",Fd-3m,cubic,-9.570786,357.328702,-0.206265,Co2Mo4N,0.000000,0.0000,9.584776,...,The material Co2Mo4N with high electrical cond...,Co2Mo4N,"[[0, 22, 0, 21, 0, 17, 0, 2, 0, 3, 0, 1, 0, 10...","[[-0.3580176532268524, 1.3459880352020264, 1.9...",28,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The material with the database ID 113703 has a...,"[This material is cubic., This material is not...","[This material is composed of ['Molybdenum', '..."
113704,"{'abc': [13.38737, 16.5407, 19.4082], 'angles'...",Pbcn,orthorhombic,-4.051578,4297.685141,-1.525848,TeAs(Se2F3)2,0.086280,1.4334,3.909250,...,TeAs(Se2F3)2 has interesting electronic and op...,TeAsSe4F6,"[[0, 45, 0, 51, 0, 50, 0, 98, 0, 183, 0, 136, ...","[[-1.1632286310195923, 0.5010343194007874, -2....",192,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The substance stored in the database has a pre...,"[This material is orthorhombic., This material...","[This material is composed of ['Tellurium', 'F..."
113705,"{'abc': [5.420702, 5.420702, 5.73608], 'angles...",C2/m,monoclinic,-5.235020,165.406146,-1.202585,Pb2O3,0.209392,0.0000,9.284181,...,Pb2O3 is an unstable compound that is not comm...,Pb2O3,"[[0, 7, 0, 4, 0, 8, 0, 9, 0, 6, 0, 5, 0, 3, 0,...","[[-1.6124303340911865, 1.3321336507797241, -0....",10,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The material in this database has a pretty for...,"[This material is monoclinic., This material i...","[This material is composed of ['Oxygen', 'Lead..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126330,"{'abc': [5.17955, 5.179548, 15.91706], 'angles...",Cmcm,orthorhombic,-5.416528,238.120897,-0.574610,Fe2O3,1.332748,0.0000,4.454351,...,"Based on the properties described, the materia...",Fe2O3,"[[0, 11, 0, 15, 0, 15, 0, 14, 0, 9, 0, 9, 0, 0...","[[-1.0046907663345337, 0.30613380670547485, 1....",20,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...",[0],The substance in question is an orthorhombic m...,"[This material is orthorhombic., This material...","[This material is composed of ['Oxygen', 'Iron..."
126331,"{'abc': [7.898652, 7.898655, 17.29874], 'angle...",C2,monoclinic,-6.171699,691.809324,-1.918218,Bi6P2O15,0.104265,0.0000,7.468825,...,Bi6P2O15 is a type of bismuth-based phosphate ...,Bi6P2O15,"[[0, 28, 0, 32, 0, 24, 0, 42, 0, 42, 0, 25, 0,...","[[-0.07936850190162659, 0.4460204243659973, -2...",46,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The material with the formula Bi6P2O15 has a c...,"[This material is monoclinic., This material i...","[This material is composed of ['Bismuth', 'Pho..."
126332,"{'abc': [5.937475,

In [14]:
df['structure_question_list'][113704]

array(['This material is orthorhombic.',
       'This material is not tetragonal.',
       'This material is not hexagonal.', 'This material is not cubic.',
       'This material is not trigonal.',
       'This material is not monoclinic.',
       'This material is not triclinic.'], dtype=object)

In [1]:
import pandas as pd

In [2]:
pdf_0 = pd.read_parquet(f'/mnt/hdd1/SESPapers/multi-label/materials_paper_split_{0}.parquet')
pdf_0

,text,doi
0,First-principles molecular simulations of Li d...,10.1016/j.commatsci.2015.05.022
1,Few‐Layered SnS2 on Few‐Layered Reduced Graphe...,10.1002/adfm.201402833
2,Effect of Processing Conditions of 75Li2S-25P2...,10.1016/j.electacta.2017.03.200
3,"Electrical conductivity, electrical modulus, a...",10.1007/s11581-011-0588-5
4,High Performance 3D Si/Ge Nanorods Array Anode...,10.1002/adfm.201403648
...,...,...
87844,Platinum-catalyzed high temperature oxidation ...,10.1016/j.corsci.2007.03.010
87845,Promotion effect of nickel loaded on CdS for p...,10.1016/j.apsusc.2014.08.053
87846,PEG-assisted synthesis of crystal TiO2 nanowir...,10.1016/j.cej.2015.01.006
87847,“Four Birds with One Stone”: Synthesis of Nano...,10.1002/anie.200602093


In [5]:
pdf_0['text'][70906]

'Development of a used fuel cladding damage model incorporating circumferential and radial hydride responses\nABSTRACT\nAt the completion of the fuel drying process, used fuel Zry4 cladding typically exhibits a significant population of δ -hydride inclusions. These inclusions are in the form of small platelets that are generally oriented both circumferentially and radially within the cladding material. There is concern that radially-oriented hydride inclusions may weaken the cladding material and lead to issues during used fuel storage and transportation processes. A high fidelity model of the mechanical behavior of hydrides has utility in both designing fuel cladding to be more resistant to this hydride-induced weakening and also in suggesting modifications to drying, storage, and transport operations to reduce the impact of hydride formation and/or the avoidance of loading scenarios that could overly stress the radial inclusions.We develop a mechanical model for the Zry4-hydride syst

In [3]:
import pandas as pd
mp_papers = pd.read_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_cif_papers.parquet')
mp_papers

,reduced_formula,spg,crystal system,lattice_parameters,formula,volume,cif,doi,num_papers,edge_index,edge_attr,node_feat,y,text
0,Cs2CuSbBr6,Fm-3m,cubic,"[7.833244, 7.833245, 7.83325, 60.0, 60.0, 60.0]",Cs2CuSbBr6,339.868504,data_Cs2CuSbBr6\n_symmetry_space_group_name_H-...,10.1021/jacs.6b09645,1,"[[0, 8, 0, 9, 0, 3, 0, 3, 0, 4, 0, 4, 0, 1, 0,...","[[3.847209088050251e-16, 2.688601493835449, 3....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],We begin by probing the most energetically fav...
0,Cs2CuSbBr6,Fm-3m,cubic,"[7.833244, 7.833245, 7.83325, 60.0, 60.0, 60.0]",Cs2CuSbBr6,339.868504,data_Cs2CuSbBr6\n_symmetry_space_group_name_H-...,10.1021/jacs.6b09645,1,"[[0, 8, 0, 9, 0, 3, 0, 3, 0, 4, 0, 4, 0, 1, 0,...","[[3.847209088050251e-16, 2.688601493835449, 3....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],With the data of phase stability and electroni...
1,TiBe,Pm-3m,cubic,"[2.93071, 2.93071, 2.93071, 90.0, 90.0, 90.0]",TiBe,25.171970,data_TiBe\n_symmetry_space_group_name_H-M 'Pm...,10.1021/acssuschemeng.8b03545,1,"[[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,...","[[1.4653550386428833, -1.4653550386428833, -1....","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],Electrochemical deposition is an effective met...
2,TmNi,Pnma,orthorhombic,"[4.10291, 5.36576, 6.94965, 90.0, 90.0, 90.0]",Tm4Ni4,152.998194,data_TmNi\n_symmetry_space_group_name_H-M 'Pn...,10.1016/j.matlet.2013.11.055,1,"[[0, 1, 0, 1, 0, 4, 0, 4, 0, 6, 0, 6, 0, 5, 0,...","[[-0.5171929597854614, 2.051455020904541, -1.3...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],Fig. 1(a) shows the XRD patterns for both the ...
3,TeO3,C2/c,monoclinic,"[8.826298, 8.826301, 6.01211, 75.8593, 75.8593...",Te2O6,440.410598,data_TeO3\n_symmetry_space_group_name_H-M 'C2...,10.1016/j.electacta.2012.07.042,2,"[[0, 3, 0, 3, 0, 4, 0, 1, 0, 1, 0, 6, 0, 6, 0,...","[[-0.9152435660362244, 0.7457637190818787, 0.3...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],All chemicals were from commercial sources and...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16647,NaAl11O17,R3m,trigonal,"[11.79236, 11.792361, 11.79236, 27.7342, 27.73...",NaAlO7,313.558942,data_NaAl11O17\n_symmetry_space_group_name_H-M...,10.1016/j.lithos.2019.05.013,2,"[[0, 25, 0, 20, 0, 15, 0, 24, 0, 23, 0, 18, 0,...","[[-1.416701078414917, 0.8179340958595276, -0.8...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],Fused alumina or electro-corundum is prepared ...
16648,V3S4,P6_3/m,hexagonal,"[9.08517, 9.085174, 3.25619, 90.0, 90.0, 120.0]",V6S8,232.759121,data_V3S4\n_symmetry_space_group_name_H-M 'P6...,10.1016/j.jallcom.2021.161976,1,"[[0, 9, 0, 9, 0, 10, 0, 10, 0, 13, 0, 13, 0, 4...","[[-1.1388251781463623, 1.3956764936447144, -1....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[0],The exact compositions of the synthesised vana...
16649,Fe2O3,Cmcm,orthorhombic,"[5.17955, 5.179548, 15.91706, 90.0, 90.0, 146....",Fe8O2,238.120896,data_Fe2O3\n_symmetry_space_group_name_H-M 'C...,10.1016/j.coal.2013.02.008,2,"[[0, 14, 0, 10, 0, 10, 0, 11, 0, 13, 0, 13, 0,...","[[1.2001427022730807e-17, 1.0502866506576538, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...",[0],The Pond Creek coal is a Pennsylvanian-age coa...
16649,Fe2O3,Cmcm,orthorhombic,"[5.17955, 5.179548, 15.91706, 90.0, 90.0, 146....",Fe8O2,238.120896,data_Fe2O3\n_symmetry_space_group_name_H-M 'C...,10.1016/j.oregeorev.2017.02.029,2,"[[0, 14, 0, 10, 0, 10, 0, 11, 0, 13, 0, 13, 0,...","[[1.2001427022730807e-17, 1.0502866506576538, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...",[0],The analyses of igneous rocks were performed i...


In [7]:
mp_papers = mp_papers[['reduced_formula', 'crystal system','doi']]
# deduplicate for doi
mp_papers = mp_papers.drop_duplicates(subset=['doi'])

In [9]:
mp_papers.to_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_doi_deduplicated.parquet')

In [2]:
import pandas as pd
mp_papers = pd.read_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_doi_deduplicated.parquet')

In [62]:
mp_iter = mp_papers.iterrows()

In [30]:
prompt = '''Please play a role in creating 5-choice questions.
Your goal is to extract materials engineering knowledge for a #given material# from a #given academic paper# and create a multiple-choice question that can measure understanding of an artificial intelligence model.
Please follow the steps below to make this happen:
1. List the hidden knowledge about #given material# in #given academic paper#.
2. Write questions and correct answers related to the mentioned knowledge.
3. Based on the above information, write some explanations to ensure the validity of the knowledge by adding restrictions, conditions, and relevant knowledge points.
4. Write information to explicitly focus on multi-level inferences between this knowledge.
5. Write four incorrect answers to the mentioned question and write an explanation for each choice as well.
6. Follow the form below:
#Target material#:
<#Given material#>
#Question#:
<your created question>
#Right Answer#:
1) <your right answer>
#Wrong Answer#:
2) <your wrong answer1>
3) <your wrong answer2>
4) <your wrong answer3>
5) <your wrong answer4>
#Explanation#:
<explanation for right answer>
<explanation for wrong answer1>
<explanation for wrong answer2>
<explanation for wrong answer3>
<explanation for wrong answer4>

Below is the information given to create the problem.
#Given material#:
{}
#Given academic paper#:
{}
'''

In [4]:
prompt = '''Please play a role in creating 5-narratives.
Your goal is to extract materials engineering knowledge for a #given material# from a #given academic paper# and create a True-False question that can measure understanding of an AI model.
Therefore, all the sentences in this work must include the #given material#.
Please follow the steps below to make this happen:
1. List the #Hidden Knowledge# about #given material# in #given academic paper#.
2. Write correct knowledge as a #Right Answer#. The statement must include the #Given material#. This knowledge must be reminiscent from the #given material# rather than describing experimental methods. 
3. Write four incorrect knowledge as #Wrong Answer#. The statement must include the #Given material#. They must be False even if it is not mentioned in the paper.
4. Based on the above information, write some explanations to ensure the validity of the knowledge by adding restrictions, conditions, and relevant knowledge points.
5. Write information to explicitly focus on multi-level inferences between this knowledge.
6. Follow the json form below:
"Target material": <#Given material#>
"Hidden Knowledge": <list of hidden knowledges>
"Statements": <list of statements>. The order is [#Right Answer#, #Wrong Answer1#, #Wrong Answer2#, #Wrong Answer3#, #Wrong Answer4#]
"Explanations": <list of explanations>. The order is same as "statements".

Below is the information given to create the problem.
#Given material#:
{}
#Given academic paper#:
{}
'''

In [5]:
nuclear_filter = pdf_0['doi'].str.contains('j.jnucmat')
battery_filter = pdf_0['text'].str.contains('battery')

In [71]:
mp_papers['GPT_question'] = None

In [6]:
def chatgpt(client, prompt_input):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt_input},
    ],
    response_format={'type':'json_object'}
    )
    return json.loads(response.choices[0].message.content)

In [7]:
# parse the reply_json from str to json
import json

In [18]:
item = row
# row['Hidden Knowledge_nuclear'] = reply_json['Hidden Knowledge']
# row['Statements_nuclear'] = reply_json['Statements']
# row['Explanations_nuclear'] = reply_json['Explanations']

TypeError: string indices must be integers

In [9]:
pdf_0[nuclear_filter]

,text,doi
38651,Mass changes in NSTX surface layers with Li co...,10.1016/j.jnucmat.2009.01.263
38656,Evaporated lithium surface coatings in NSTX\nA...,10.1016/j.jnucmat.2009.01.262
39460,High flux irradiations of Li coatings on polyc...,10.1016/j.jnucmat.2014.11.102
39494,Modeling the reduction of gross lithium erosio...,10.1016/j.jnucmat.2014.11.056
40212,Interatomic potential for the compound-forming...,10.1016/j.jnucmat.2014.01.037
...,...,...
87121,Effect of carbon ion irradiation on Ag diffusi...,10.1016/j.jnucmat.2015.11.017
87250,Tungsten blow-off in response to the ignition ...,10.1016/j.jnucmat.2010.08.030
87398,Hydrogen isotope retention of JT-60U W-shaped ...,10.1016/j.jnucmat.2006.06.001
87676,Molecular dynamics simulation of dislocations ...,10.1016/j.jnucmat.2013.07.059


In [7]:
from openai import OpenAI
from dotenv import load_dotenv
import os
from joblib import Parallel, delayed
import numpy as np
from tqdm import tqdm
import json
load_dotenv()
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

pdf_0_nuclear = pdf_0[nuclear_filter]
nuc_questions = []
for i, row in tqdm(mp_papers.iterrows()):
    try:
        text = pdf_0_nuclear[pdf_0_nuclear['doi']==row['doi']]['text'].iloc[0]
        prompt_input = prompt.format(row['reduced_formula'], text[:20000])
        reply_json = chatgpt(client, prompt_input)
        row['Hidden Knowledge'] = reply_json['Hidden Knowledge']
        row['Statements'] = reply_json['Statements']
        row['Explanations'] = reply_json['Explanations']
        nuc_questions.append(row)
    except :pass

439879it [1:12:33, 101.05it/s] 


In [26]:
nuc_questions = pd.DataFrame(nuc_questions)
nuc_questions['Hidden Knowledge'] = nuc_questions['Hidden Knowledge'].apply(str)
nuc_questions.to_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_nuclear_questions_0.parquet')

In [30]:
# deduplicate doi
nuc_questions = nuc_questions.drop_duplicates(subset=['doi'])
nuc_questions.to_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_nuclear_questions_0_val.parquet')

In [52]:
nuc_questions_gpt4_solving = []
for i in tqdm(range(len(nuc_questions))):
    prompt_input = 'Which of the following statements is most correct regarding the given material? Please answer in json: "number of answer": <number of answer statement>, "explanation": <your opinion>.\n'
    prompt_input += 'Given material:' + nuc_questions.iloc[i]['reduced_formula'] + '\n'
    for j, state in enumerate(nuc_questions.iloc[i]['Statements']):
        prompt_input += str(j+1) + state + '\n'    
    response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt_input},
    ],
    response_format={'type':'json_object'}
    )
    nuc_questions_gpt4_solving.append(json.loads(response.choices[0].message.content))


In [77]:
cnt = 0 
for _, solving in nuc_questions_gpt4_solving.iterrows():
    # solving['number of answer'] = int(solving['number of answer'])
    if type(solving['number of answer']) != int:
        solving['number of answer'] = 0
    if int(solving['number of answer']) == 1:
        cnt += solving['number of answer']
cnt/len(nuc_questions_gpt4_solving)*100

36.92307692307693

In [79]:
nuc_questions_gpt4_solving = pd.DataFrame(nuc_questions_gpt4_solving)
nuc_questions_gpt4_solving.to_parquet('/home/yj/PycharmProjects/MIT/CLaMP/jsons/mp_3d_2020_nuclear_questions_0_gpt4_solving.parquet')

In [20]:
from openai import OpenAI
from dotenv import load_dotenv
import os
from joblib import Parallel, delayed
import numpy as np
from tqdm import tqdm
load_dotenv()
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

pdf_0_nuclear = pdf_0[nuclear_filter]
mp_papers_chunk = np.array_split(mp_papers, 8)
def treat_chunk(chunk):
    nuc_questions = []
    for i, row in tqdm(chunk.iterrows()):
        try:
            text = pdf_0_nuclear[pdf_0_nuclear['doi']==row['doi']]['text'].iloc[0]
            prompt_input = prompt.format(row['reduced_formula'], text[:20000])
            reply_json = chatgpt(client, prompt_input)
            row['Hidden Knowledge'] = reply_json['Hidden Knowledge']
            row['Statements'] = reply_json['Statements']
            row['Explanations'] = reply_json['Explanations']
            nuc_questions.append(row)
        except : pass
    return nuc_questions
results = Parallel(n_jobs=8)(delayed(treat_chunk)(chunk) for chunk in mp_papers_chunk)

nuc_questions = pd.DataFrame(sum(results, []))
nuc_questions
    

/home/yj/anaconda3/envs/llamat/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


PicklingError: Could not pickle the task to send it to the workers.

In [64]:
row

reduced_formula                                                    U3O7
spg                                                                Immm
crystal system                                             orthorhombic
lattice_parameters    [6.569154, 6.5691559999999996, 6.5691500000000...
formula                                                            U3O7
volume                                                       117.772709
cif                   data_U3O7\n_symmetry_space_group_name_H-M  'Im...
doi                                       10.1016/j.jnucmat.2006.02.096
num_papers                                                            2
edge_index            [[0, 7, 0, 7, 0, 8, 0, 9, 0, 3, 0, 3, 0, 6, 0,...
edge_attr             [[-1.8936500549316406, 1.1925625801086426, -3....
node_feat             [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
y                                                                   [0]
text                  The oxidation of UO2 into U3O8 has been st

In [65]:

response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt_input},
  ],
  response_format={'type':'json_object'}
)
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-8JWo0xqxm287cDnlIwQlYNYUjseCE', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='{\n  "Target material": "U3O7",\n  "Hidden Knowledge": [\n    "The oxidation of sintered UO2 pellets and powder into U3O8 has been studied at 370 °C under controlled oxygen partial pressures.",\n    "The oxidation of UO2 into U3O8 is at least a two-step reaction, usually written as UO2 -> U4O9/U3O7 -> U3O8.",\n    "The temperatures leading to a significant oxidation of sintered UO2 pellets are higher (about 300–350 °C) and it seems that if only a thin layer of U3O7 is formed on the surface of the pellets, then the oxidation to U3O8 proceeds according to only sigmoidal reaction curves.",\n    "There is no universal agreement in the case of U4O9/U3O7 formation on powders.",\n    "The reaction of UO2 to U3O8 proceeds in a pseudo steady state; and a mean value of activation energy for the formation of U3O8 of 103 kJ mol−1 was estimated."\n  ],\n 

{
  "Target material": "U3O7",
  "Hidden Knowledge": [
    "The oxidation of sintered UO2 pellets and powder into U3O8 has been studied at 370 °C under controlled oxygen partial pressures.",
    "The oxidation of UO2 into U3O8 is at least a two-step reaction, usually written as UO2 -> U4O9/U3O7 -> U3O8.",
    "The temperatures leading to a significant oxidation of sintered UO2 pellets are higher (about 300–350 °C) and it seems that if only a thin layer of U3O7 is formed on the surface of the pellets, then the oxidation to U3O8 proceeds according to only sigmoidal reaction curves.",
    "There is no universal agreement in the case of U4O9/U3O7 formation on powders.",
    "The reaction of UO2 to U3O8 proceeds in a pseudo steady state; and a mean value of activation energy for the formation of U3O8 of 103 kJ mol−1 was estimated."
  ],
  "Statements": [
    "The oxidation of U3O7 into U3O8 has been studied at 370 °C under controlled oxygen partial pressures.",
    "The formation of U3O7 co